# Steam Game Dataset Analysis

## Introduction

This is a practice dataset analysis pilot project for my High School Computer Science course. Throughout this notebook, I'll be using Steam Store data that was scraped and uploaded onto Kaggle. In the future, I'll likely do another trial by scraping and cleaning out my own data.

In [24]:
# import necessary libraries
import io

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [6]:
sd = pd.read_json('https://media.githubusercontent.com/media/1metropolis/steam-store-analysis/refs/heads/main/rawdata/steam-data.json')

In [25]:
sd.head()

,20200,655370,1732930,1355720,1139950,1469160,1659180,1968760,1178150,320150,...,3309950,1989760,3136850,1721060,3593600,3600970,3543710,3265370,3423620,3183790
name,Galactic Bowling,Train Bandit,Jolt Project,Henosis™,Two Weeks in Painland,Wartune Reborn,TD Worlds,Legend of Rome - The Wrath of Mars,MazM: Jekyll and Hyde,Deadlings: Rotten Edition,...,PlanetX,Pax Augusta,Kafkaesque: The Nightmare Trial,Mandragora: Whispers of the Witch Tree,Starry Trace,Paragon Of Time,A Few Days With : Hazel,MosGhost,AccuBow VR,Defense Of Fort Burton
release_date,"Oct 21, 2008","Oct 12, 2017","Nov 17, 2021","Jul 23, 2020","Feb 3, 2020","Feb 26, 2021","Jan 9, 2022","May 5, 2022","Apr 2, 2020","Nov 11, 2014",...,"Apr 18, 2025","Apr 17, 2025","Apr 17, 2025","Apr 17, 2025","Apr 14, 2025","Apr 10, 2025","Apr 11, 2025","Apr 1, 2025","Mar 11, 2025","Apr 10, 2025"
required_age,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
price,19.99,0.99,4.99,5.99,0.0,0.0,10.99,9.99,14.99,3.99,...,0.0,19.99,2.39,39.99,1.21,2.99,2.69,7.99,0.0,9.99
dlc_count,0,0,0,0,0,0,1,0,0,0,...,0,0,0,7,0,0,0,0,0,0


The data from the Steam Storefront is listed with the game ID as the column, and different attributes in the rows. The following attributes are available for each game entry:

In [34]:
sd.index

Index(['name', 'release_date', 'required_age', 'price', 'dlc_count',
       'detailed_description', 'about_the_game', 'short_description',
       'reviews', 'header_image', 'website', 'support_url', 'support_email',
       'windows', 'mac', 'linux', 'metacritic_score', 'metacritic_url',
       'achievements', 'recommendations', 'notes', 'supported_languages',
       'full_audio_languages', 'packages', 'developers', 'publishers',
       'categories', 'genres', 'screenshots', 'movies', 'user_score',
       'score_rank', 'positive', 'negative', 'estimated_owners',
       'average_playtime_forever', 'average_playtime_2weeks',
       'median_playtime_forever', 'median_playtime_2weeks', 'peak_ccu', 'tags',
       'discount'],
      dtype='object')

## Cleaning Data

## Modification

## Analysis

### Section 1

### Section 2

### Section 3